In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run VAE models systematically

## Imports

In [104]:
import pandas as pd
import numpy as np
import itertools


## Create table of all VAE model training settings

Parameters for:
- Biological dataset generation
- Training data
    - Input
    - Output 
- Model architecture
- Training hyperparameters

In [105]:
'USE_SIGMOID_DECODER'.lower()

'use_sigmoid_decoder'

### Initial parameters

In [112]:
['PREP_X_CATEGORICAL_ONEHOT'.lower(),
'PREP_Y_CATEGORICAL_ONEHOT'.lower(),
'PREP_X_CATEGORICAL_NBINS'.lower(),
'PREP_Y_CATEGORICAL_NBINS'.lower()]

['prep_x_categorical_onehot',
 'prep_y_categorical_onehot',
 'prep_x_categorical_nbins',
 'prep_y_categorical_nbins']

In [ ]:
hpos_architecture = {
    'seed_arch': 1,
    'hidden_size': 32,
    'enc_layers': [64, 64, 64],
    'dec_layers': [64, 64, 64],
    'model': 'CVAE',
    'use_sigmoid_decoder': False,
    'init_enc': 'HeNormal',
    'init_dec': 'HeNormal',
    'init_model_with_random': True,
    'activation': 'leaky_relu',
}




hpos_training = {
    'batch_size': 128,
    'epochs': 2000,
    'learning_rate': 1e-1,
    'learning_rate_sched': 'cosine_decay',
    'loss_func': 'mse_loss',
    'use_dropout': False,
    'dropout_rate': 0.1,
    'use_l2_reg': False,
    'l2_reg_alpha': 0.01,
    'use_kl_div': True,
    'kl_weight': 0.00025,  # inspired by https://github.com/elttaes/VAE-MNIST-Haiku-Jax/blob/main/cVAE_mnist.ipynb
    'use_warmup': True,
    'warmup_epochs': 20
}
hpos_training['print_every'] = hpos_training['epochs'] // 100

hpos_optimization = {
    'seed_opt': 1,
    'opt_method': 'adam',
    'opt_metric': 'mean_absolute_error',
    'opt_mode': 'min',
    'opt_patience': 100,
    'opt_factor': 0.5,
    'opt_min_lr': 1e-6,
    'opt_min_delta': 1e-4
}

hpos_dataset = {
    'seed_dataset': 1,
    'include_diffs': False,
    'objective_col': 'adaptability',
    'output_species': ['RNA_2'],
    # 'total_ds': None,   # TO BE RECORDED
    'total_ds_max': 3e6,
    'train_split': 0.8,
    'x_type': 'energies',
    # XY filtering:
    'filt_x_nans': True,
    'filt_y_nans': True,
    'filt_sensitivity_nans': True,
    'filt_precision_nans': True,
    'filt_n_same_x_max': 100,
    'filt_n_same_x_max_bins': 500,
    # XY preprocessing:
    'prep_x_standardise': False,
    'prep_y_standardise': False,
    'prep_x_minmax': False,
    'prep_y_minmax': False,
    'prep_x_robust_scaling': True,
    'prep_y_robust_scaling': True,
    'prep_x_logscale': False,
    'prep_y_logscale': False,
    'prep_x_categorical': False,
    'prep_y_categorical': False,
    'prep_x_categorical_onehot': False,
    'prep_y_categorical_onehot': False,
    'prep_x_categorical_nbins': 10,
    'prep_y_categorical_nbins': 10,
    'prep_x_categorical_method': 'quantile',
    'prep_y_categorical_method': 'quantile',
    'prep_x_neg': False,
    'prep_y_neg': False,
}

hpos_biological = {
    'filenames_train_config': ['EvoScaper/data/raw/summarise_simulation/2024_12_05_210221/ensemble_config.json'], 
    'filenames_train_table': ['EvoScaper/data/raw/summarise_simulation/2024_12_05_210221/tabulated_mutation_info.csv'],
    'filenames_verify_config': ['EvoScaper/data/raw/summarise_simulation/2024_11_21_160955/ensemble_config.json'], 
    'filenames_verify_table': ['EvoScaper/data/raw/summarise_simulation/2024_11_21_160955/tabulated_mutation_info.csv'],
    'n_species': 3,
    'sequence_length': 20,
    'signal_function': 'step_function',
    'signal_target': 2,
    'starting_copynumbers_input': [200],
    'starting_copynumbers_output': [200],
    'starting_copynumbers_other': [200],
    'association_binding_rate': 1000000,
    'include_prod_deg': False,
}

info_to_be_recorded = {
    'filename_saved_model': 'TO_BE_RECORDED',
    'total_ds': 'TO_BE_RECORDED',
    'n_batches': 'TO_BE_RECORDED',
    'R2_train': 'TO_BE_RECORDED',
    'R2_test': 'TO_BE_RECORDED',
    'conditionality_fidelity': 'TO_BE_RECORDED',
    'n_layers_enc': 'TO_BE_RECORDED',
    'n_layers_dec': 'TO_BE_RECORDED',
}



In [107]:
df_hpos = pd.concat([pd.DataFrame.from_dict(hpos, orient='index').T for hpos in [hpos_architecture, hpos_training, hpos_optimization, hpos_dataset]], axis=1)
assert df_hpos.columns.duplicated().sum() == 0, 'Change some column names, there are duplicates'
basic_setting = df_hpos.copy()
df_hpos

,seed_arch,hidden_size,enc_layers,dec_layers,model,use_sigmoid_decoder,init_enc,init_dec,batch_size,epochs,...,prep_x_minmax,prep_y_minmax,prep_x_robust_scaling,prep_y_robust_scaling,prep_x_logscale,prep_y_logscale,prep_x_categorical,prep_y_categorical,prep_x_neg,prep_y_neg
0,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False


### All parameters

In [108]:

hpos_to_vary_from_og = {
    'total_ds_max': [1e4, 5e4, 1e5, 5e5, 1e6, 5e6],
    'seed_arch': [1, 2, 3, 4, 5],
}
hpos_to_vary_together = {
    'hidden_size': [16, 32, 64, 128, 256, 512],
    'objective_col': ['adaptability', 'sensitivity_wrt_species-6'],
    'x_type': ['energies', 'binding_rates_dissociation'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
}

df_hpos.loc[df_hpos['objective_col'] == 'sensitivity_wrt_species-6', 'prep_y_logscale'] = True

In [109]:
for h, v in hpos_to_vary_from_og.items():
    df_hpos = pd.concat([df_hpos] + [basic_setting.assign(**{h: vv}) for vv in v], ignore_index=True)
df_hpos

,seed_arch,hidden_size,enc_layers,dec_layers,model,use_sigmoid_decoder,init_enc,init_dec,batch_size,epochs,...,prep_x_minmax,prep_y_minmax,prep_x_robust_scaling,prep_y_robust_scaling,prep_x_logscale,prep_y_logscale,prep_x_categorical,prep_y_categorical,prep_x_neg,prep_y_neg
0,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
1,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
2,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
3,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
4,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
5,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
6,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
7,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
8,2,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False
9,3,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,128,1500,...,False,False,True,True,False,False,False,False,False,False


In [110]:
keys_vary_together = sorted(hpos_to_vary_together.keys())
for v in itertools.product(*[hpos_to_vary_together[h] for h in keys_vary_together]):
    curr = basic_setting.assign(**{h: vv for h, vv in zip(keys_vary_together, v)})
    df_hpos = pd.concat([df_hpos, curr], ignore_index=True)
print('All good if these are equal: ', len(df_hpos), len(list(itertools.product(*[hpos_to_vary_together[h] for h in keys_vary_together]))) + np.sum([len(v) for v in hpos_to_vary_from_og.values()]) + 1)

All good if these are equal:  84 84


# Use table to create dataset for training